In [11]:
import numpy as np
import torch
import scipy
from torch import nn
from torch.nn.functional import elu
import torchvision.transforms as transform
import braindecode 
from braindecode.models import *
from braindecode.models.modules import Expression
from braindecode.models.functions import squeeze_final_output
from braindecode.datasets import BaseDataset, BaseConcatDataset,create_from_X_y
from braindecode.models.util import to_dense_prediction_model, get_output_shape
import pandas as pd
import resampy
from skorch.dataset import Dataset
from skorch.callbacks import Checkpoint,ProgressBar
from skorch.helper import predefined_split
from config import *
from dataset import *
from braindecode.preprocessing import create_fixed_length_windows
from mne import set_log_level
set_log_level(False)
device = 'cuda' if cuda else 'cpu'

In [12]:
preproc_functions = []
preproc_functions.append( lambda data, fs: (data[:, int(sec_to_cut * fs):-int(sec_to_cut * fs)], fs))
preproc_functions.append(lambda data, fs: (data[:, :int(duration_recording_mins * 60 * fs)], fs))
if max_abs_val is not None:
    preproc_functions.append(lambda data, fs:(np.clip(data, -max_abs_val, max_abs_val), fs))
preproc_functions.append(lambda data, fs: (resampy.resample(data, fs,sampling_freq,axis=1,filter='kaiser_fast'),sampling_freq))
if divisor is not None:
    preproc_functions.append(lambda data, fs: (data / divisor, fs))
dataset = DiagnosisSet(n_recordings=n_recordings,
                           max_recording_mins=max_recording_mins,
                           preproc_functions=preproc_functions,
                           data_folders=data_folders,
                           train_or_eval='train',
                           sensor_types=sensor_types)
if test_on_eval:
    test_dataset = DiagnosisSet(n_recordings=n_recordings,
                           max_recording_mins=max_recording_mins,
                           preproc_functions=preproc_functions,
                           data_folders=data_folders,
                           train_or_eval='eval',
                           sensor_types=sensor_types)

In [13]:
X,y=dataset.load()
if test_on_eval:
    test_x,test_y=test_dataset.load()

In [14]:
del divisor,max_abs_val,sec_to_cut,duration_recording_mins,preproc_functions
def create_set(X, y, inds):
    """
    X list and y nparray
    :return: 
    """
    new_X = []
    for i in inds:
        new_X.append(X[i])
    new_y = y[inds]
    return (new_X, new_y)
#Use of TrainValidTestSplitter is not necessary in newer versions of braindecode
class TrainValidSplitter(object):
    def __init__(self, n_folds, i_valid_fold, shuffle):
        self.n_folds = n_folds
        self.i_valid_fold = i_valid_fold
        self.rng = np.random.RandomState(39483948)
        self.shuffle = shuffle

    def split(self, X, y):
        if len(X) < self.n_folds:
            raise ValueError("Less Trials: {:d} than folds: {:d}".format(
                len(X), self.n_folds
            ))
        indices=np.arange(len(y))
        #Compared to paper, the valid set will be unbalanced
        batch_size=len(X)//self.n_folds
        if self.shuffle:
            self.rng.shuffle(indices)
        valid_inds=indices[self.i_valid_fold*batch_size:(self.i_valid_fold+1)*batch_size]
        train_inds = np.setdiff1d(indices,valid_inds)
        train_set = create_set(X, y, train_inds)
        valid_set = create_set(X, y, valid_inds)
        return train_set, valid_set

In [15]:
if test_on_eval==False:
    splitter=TrainValidSplitter(n_folds,i_test_fold,True)
    train_set,valid_set=splitter.split(X,y)
    del X,y
    X,y=train_set
    valid_X,valid_y=valid_set
    del n_folds,i_test_fold,train_set,valid_set

In [16]:
ch_names=['A1', 'A2', 'C3', 'C4', 'CZ', 'F3', 'F4', 'F7', 'F8', 'FP1','FP2', 'FZ', 'O1', 'O2','P3', 'P4', 'PZ', 'T3', 'T4', 'T5', 'T6']
#we take a 20 second stride as 1 sample and 1 second stride takes too long
stride=sampling_freq*10
train_set=create_from_X_y(X,y,sfreq=sampling_freq,drop_last_window=True,ch_names=ch_names,window_size_samples=input_time_length,
                       window_stride_samples=stride)
if test_on_eval==False:
    valid_set=create_from_X_y(valid_X,valid_y,sfreq=sampling_freq,drop_last_window=True,ch_names=ch_names,window_size_samples=input_time_length,
                        window_stride_samples=stride)
    del valid_X,valid_y
elif test_on_eval:
    test_set=create_from_X_y(test_x,test_y,sfreq=sampling_freq,drop_last_window=True,ch_names=ch_names,window_size_samples=input_time_length,
                        window_stride_samples=stride)
    del test_x,test_y
del stride,ch_names,X,y

In [17]:
n_classes = 2
if model_name=="shallow":
    optimizer_lr = 0.0000625
    optimizer_weight_decay = 0
    #The final conv length is auto to ensure that output will give two values for single EEG window
    model = ShallowFBCSPNet(n_chans,
                                    n_classes,
                                    n_filters_time=n_start_chans,
                                    n_filters_spat=n_start_chans,
                                    input_window_samples=input_time_length,
                                    final_conv_length='auto',)
    test=torch.ones(size=(7,21,6000))
    out=model.forward(test)
    print(out.shape)
elif model_name=="deep":
    optimizer_lr = init_lr
    optimizer_weight_decay = 0
    model = Deep4Net(n_chans, n_classes,
                         n_filters_time=n_start_chans,
                         n_filters_spat=n_start_chans,
                         input_window_samples=input_time_length,
                         n_filters_2 = int(n_start_chans * n_chan_factor),
                         n_filters_3 = int(n_start_chans * (n_chan_factor ** 2.0)),
                         n_filters_4 = int(n_start_chans * (n_chan_factor ** 3.0)),
                         final_conv_length='auto',
                        stride_before_pool=True)
    test=torch.ones(size=(7,21,6000,1))
    out=model.forward(test)
    print(out.shape)
elif model_name=="deep_smac":
    optimizer_lr = init_lr
    if model_name == 'deep_smac':
            do_batch_norm = False
    else:
        assert model_name == 'deep_smac_bnorm'
        do_batch_norm = True
    double_time_convs = False
    drop_prob = 0.244445
    filter_length_2 = 12
    filter_length_3 = 14
    filter_length_4 = 12
    filter_time_length = 21
    #final_conv_length = 1
    first_nonlin = elu
    first_pool_mode = 'mean'
    later_nonlin = elu
    later_pool_mode = 'mean'
    n_filters_factor = 1.679066
    n_filters_start = 32
    pool_time_length = 1
    pool_time_stride = 2
    split_first_layer = True
    n_chan_factor = n_filters_factor
    n_start_chans = n_filters_start
    model = Deep4Net(n_chans, n_classes,
            n_filters_time=n_start_chans,
            n_filters_spat=n_start_chans,
            input_window_samples=input_time_length,
            n_filters_2=int(n_start_chans * n_chan_factor),
            n_filters_3=int(n_start_chans * (n_chan_factor ** 2.0)),
            n_filters_4=int(n_start_chans * (n_chan_factor ** 3.0)),
            final_conv_length='auto',
            batch_norm=True,
            drop_prob=drop_prob,
            filter_length_2=filter_length_2,
            filter_length_3=filter_length_3,
            filter_length_4=filter_length_4,
            filter_time_length=filter_time_length,
            first_conv_nonlin=first_nonlin,
            first_pool_mode=first_pool_mode,
            later_conv_nonlin=later_nonlin,
            later_pool_mode=later_pool_mode,
            pool_time_length=pool_time_length,
            pool_time_stride=pool_time_stride,
            split_first_layer=split_first_layer,
            stride_before_pool=True)
    test=torch.ones(size=(6,21,6000,1))
    out=model.forward(test)
    print(out.shape)
#Works properly, fit the hybrid cnn
elif model_name=="hybrid":
    optimizer_lr = init_lr
    optimizer_weight_decay = 0
    #The final conv length is auto to ensure that output will give two values for single EEG window
    model = HybridNet(n_chans, n_classes,input_window_samples=input_time_length,)
    test=torch.ones(size=(2,21,6000))
    out=model.forward(test)
    out_length=out.shape[2]
    model.final_conv=nn.Conv2d(100,n_classes,(out_length,1),bias=True,)
    model=nn.Sequential(model,Expression(torch.squeeze))
    out=model.forward(test)
    print(out.shape)
    del out_length
elif model_name=="TCN":
    import warnings
    #This disables the warning of the dropout2d layers receiving 3d input
    warnings.filterwarnings("ignore")
    optimizer_lr = init_lr
    optimizer_weight_decay = 0
    n_blocks=7
    n_filters=32
    kernel_size=24
    drop_prob = 0.3
    add_log_softmax=False
    x=TCN(n_chans,n_classes,n_blocks,n_filters,kernel_size,drop_prob,add_log_softmax)
    test=torch.ones(size=(7,21,6000,1))
    out=x.forward(test)
    print(out.shape)
    out_length=out.shape[2]
    #There is no hyperparameter where output of TCN is (Batch_Size,Classes) when input is (Batch_Size,21,6000) so add new layers to meet size
    model=nn.Sequential(x,nn.Conv1d(n_classes,n_classes,out_length,bias=True,),Expression(torch.squeeze),nn.LogSoftmax(dim=1))
    out=model.forward(test)
    print(out.shape)
    del out_length,x
if cuda:
    model.cuda()
del test,out

torch.Size([6, 2])


In [13]:
model

Deep4Net(
  (ensuredims): Ensure4d()
  (dimshuffle): Expression(expression=transpose_time_to_spat) 
  (conv_time): Conv2d(1, 32, kernel_size=(21, 1), stride=(1, 1))
  (conv_spat): Conv2d(32, 32, kernel_size=(1, 21), stride=(2, 1), bias=False)
  (bnorm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_nonlin): Expression(expression=elu) 
  (pool): AvgPool2dWithConv()
  (pool_nonlin): Expression(expression=identity) 
  (drop_2): Dropout(p=0.244445, inplace=False)
  (conv_2): Conv2d(32, 53, kernel_size=(12, 1), stride=(2, 1), bias=False)
  (bnorm_2): BatchNorm2d(53, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (nonlin_2): Expression(expression=elu) 
  (pool_2): AvgPool2dWithConv()
  (pool_nonlin_2): Expression(expression=identity) 
  (drop_3): Dropout(p=0.244445, inplace=False)
  (conv_3): Conv2d(53, 90, kernel_size=(14, 1), stride=(2, 1), bias=False)
  (bnorm_3): BatchNorm2d(90, eps=1e-05, momentum=0.1, affine=True, track_runni

In [18]:
monitor = lambda net: any(net.history[-1, ('valid_accuracy_best','valid_f1_best','valid_loss_best')])
cp=Checkpoint(monitor='valid_f1_best',dirname='model',f_params=f'{model_name}best_param.pkl',
               f_optimizer=f'{model_name}best_opt.pkl', f_history=f'{model_name}best_history.json')
if test_on_eval==False:
    classifier = braindecode.EEGClassifier(
        model,
        criterion=torch.nn.NLLLoss,
        optimizer=torch.optim.AdamW,
        train_split=predefined_split(valid_set),
        optimizer__lr=optimizer_lr,
        #optimizer__weight_decay=optimizer_weight_decay,
        iterator_train__shuffle=True,
        batch_size=batch_size,
        device=device,
        callbacks=["accuracy","f1",cp],
        warm_start=True,
        )
elif test_on_eval:
    classifier = braindecode.EEGClassifier(
        model,
        criterion=torch.nn.NLLLoss,
        optimizer=torch.optim.AdamW,
        train_split=predefined_split(test_set),
        optimizer__lr=optimizer_lr,
        #optimizer__weight_decay=optimizer_weight_decay,
        iterator_train__shuffle=True,
        batch_size=batch_size,
        device=device,
        callbacks=["accuracy","f1",cp],
        warm_start=True,
        )
classifier.initialize()
del model

In [9]:
#Loads Phase 1 parameters and fit them further in phase 2
path=f'{model_name}'
if test_on_eval:
    classifier.load_params(
        f_params=f'model/{path}_param.pkl', f_optimizer=f'model/{path}_opt.pkl', f_history=f'model/{path}_history.json')
    print("Paramters Loaded")
    path=f'{model_name}II'

Paramters Loaded


In [19]:
if test_on_eval:
    path=f'{model_name}II'
elif test_on_eval==False:
    path=f'{model_name}'
try:
    classifier.load_params(
        f_params=f'model/{path}_param.pkl', f_optimizer=f'model/{path}_opt.pkl', f_history=f'model/{path}_history.json')
    print("Paramters Loaded")
except:
    pass

Paramters Loaded


In [10]:
#Shows the history of training the neural network
classifier.history_

[{'batches': [{'train_loss': 1.0325459241867065, 'train_batch_size': 64},
   {'train_loss': 0.7449325323104858, 'train_batch_size': 64},
   {'train_loss': 1.071652889251709, 'train_batch_size': 64},
   {'train_loss': 1.6339584589004517, 'train_batch_size': 64},
   {'train_loss': 1.8020044565200806, 'train_batch_size': 64},
   {'train_loss': 1.3968936204910278, 'train_batch_size': 64},
   {'train_loss': 2.063340902328491, 'train_batch_size': 64},
   {'train_loss': 1.918667197227478, 'train_batch_size': 64},
   {'train_loss': 2.0073046684265137, 'train_batch_size': 64},
   {'train_loss': 0.7767689824104309, 'train_batch_size': 64},
   {'train_loss': 1.3427547216415405, 'train_batch_size': 64},
   {'train_loss': 0.7916788458824158, 'train_batch_size': 64},
   {'train_loss': 0.8703295588493347, 'train_batch_size': 64},
   {'train_loss': 1.1513466835021973, 'train_batch_size': 64},
   {'train_loss': 1.2375996112823486, 'train_batch_size': 64},
   {'train_loss': 0.8229421973228455, 'train_ba

In [20]:
classifier.fit(train_set,y=None,epochs=3)

  epoch    train_accuracy    train_f1    train_loss    valid_accuracy    valid_f1    valid_loss    cp        dur
-------  ----------------  ----------  ------------  ----------------  ----------  ------------  ----  ---------
     13            0.9094      0.6257        0.1198            0.6064      0.3072        2.5788        1115.2896
     14            0.2178      0.2983        0.1067            0.4542      0.6188        8.9926     +  1193.8068
     15            0.2604      0.3101        0.0974            0.4705      0.6208        5.5341     +  1135.8936


<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=Deep4Net(
    (ensuredims): Ensure4d()
    (dimshuffle): Expression(expression=transpose_time_to_spat) 
    (conv_time): Conv2d(1, 32, kernel_size=(21, 1), stride=(1, 1))
    (conv_spat): Conv2d(32, 32, kernel_size=(1, 21), stride=(2, 1), bias=False)
    (bnorm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_nonlin): Expression(expression=elu) 
    (pool): AvgPool2dWithConv()
    (pool_nonlin): Expression(expression=identity) 
    (drop_2): Dropout(p=0.244445, inplace=False)
    (conv_2): Conv2d(32, 53, kernel_size=(12, 1), stride=(2, 1), bias=False)
    (bnorm_2): BatchNorm2d(53, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (nonlin_2): Expression(expression=elu) 
    (pool_2): AvgPool2dWithConv()
    (pool_nonlin_2): Expression(expression=identity) 
    (drop_3): Dropout(p=0.244445, inplace=False)
    (conv_3): Conv2d(53, 90, kernel_size=(14, 1), st

In [21]:
classifier.save_params(
    f_params=f'model/{path}_param.pkl', f_optimizer=f'model/{path}_opt.pkl', f_history=f'model/{path}_history.json')
#torch.save({"model":classifier.module_.state_dict(),"optimizer":classifier.optimizer_.state_dict()}, path)

In [ ]:
if test_on_eval==False:
    pred_labels=classifier.predict(valid_set)
    actual_labels=[label[1] for label in valid_set]
elif test_on_eval:
    pred_labels=classifier.predict(test_set)
    actual_labels=[label[1] for label in test_set]
actual_labels=np.array(actual_labels)
accuracy=np.mean(pred_labels==actual_labels)
print(f"Accuracy:{accuracy}")
tp=np.sum(pred_labels*actual_labels)
precision=tp/np.sum(pred_labels)
recall=tp/np.sum(actual_labels)
f1=2*precision*recall/(precision+recall)
print(f"F1-Score:{f1}")

In [ ]:
#Test the model on proper test set according to paper
if test_on_eval:
    try:
        del train_set,test_set
    except:
        pass
    test_x,test_y=test_dataset.load()
    ch_names=['A1', 'A2', 'C3', 'C4', 'CZ', 'F3', 'F4', 'F7', 'F8', 'FP1','FP2', 'FZ', 'O1', 'O2','P3', 'P4', 'PZ', 'T3', 'T4', 'T5', 'T6']
    #Stride between windows is set to sampling frequency as written in paper
    test_set=create_from_X_y(test_x,test_y,sfreq=sampling_freq,drop_last_window=True,ch_names=ch_names,window_size_samples=input_time_length,
                        window_stride_samples=sampling_freq)
    del test_x,test_y

In [ ]:
if test_on_eval:    
    pred_labels=classifier.predict(test_set)
    actual_labels=[label[1] for label in test_set]
    actual_labels=np.array(actual_labels)
    accuracy=np.mean(pred_labels==actual_labels)
    print(f"Accuracy:{accuracy}")
    tp=np.sum(pred_labels*actual_labels)
    precision=tp/np.sum(pred_labels)
    recall=tp/np.sum(actual_labels)
    f1=2*precision*recall/(precision+recall)
    print(f"F1-Score:{f1}") 

In [ ]:
#This will load the model and parameters and then replace it with one whose classification layer is removed
from skorch import NeuralNet
network=NeuralNet(module=model,criterion=torch.nn.modules.loss.NLLLoss,batch_size=batch_size,device=device)
network.initialize()
network.load_params(
    f_params=f'model/{model_name}best_param.pkl', f_optimizer=f'model/{model_name}best_opt.pkl', f_history=f'model/{model_name}best_history.json')
print("Paramters Loaded")
network.module_=torch.nn.Sequential(*(list(network.module_.children())[:-3]),nn.modules.Flatten())

In [ ]:
network.module_

In [ ]:
test=torch.ones(size=(2,21,6000))
network.predict(test).shape

In [ ]:
#Loads dataset, finds smallest trial, with this, we find number of windows using stride and convert it to array of windows of trials
#shape is (no_of_trials,no_of_windows,channels,input_time_length) in the end
X,y=dataset.load()
min_shape=X[0].shape[1]
for arr in X:
    if min_shape>arr.shape[1]:
        min_shape=arr.shape[1]
del arr
print(min_shape)

In [ ]:
#30 second stride between windows
stride=sampling_freq*10
no_of_windows=((min_shape-input_time_length)//stride)
#To make the features for the LSTM, we will make all the trials of same length as smallest to allow batch training
trials=[]
labels=[]
for i in range(len(X)):
    windows=[]
    for j in range(no_of_windows):
        windows.append(X[i][:,j*stride:j*stride+input_time_length])
        #The LSTM will be trained on 3 contiguous crops of trials, as in, 3 continuous windows will be fed to LSTM
        if len(windows)==3:
            trials.append(windows)
            labels.append(y[i])
            windows=[]
trials=np.array(trials)
labels=np.array(labels)
del windows,X,y

In [ ]:
#This will calculate the features before classification layer
features=[]
for i in range(len(trials)):
    out=network.predict(trials[i])
    features.append(out)
features=np.asarray(features)
del trials,out

In [ ]:
#This saves the features along with labels of each trial in a .mat file
scipy.io.savemat("E:/train_features.mat",{"x":features,"y":labels})
del features,labels

In [ ]:
test_x,test_y=test_dataset.load()
min_shape=48000
stride=sampling_freq*10
no_of_windows=((min_shape-input_time_length)//stride)
trials=[]
labels=[]
for i in range(len(test_x)):
    windows=[]
    for j in range(no_of_windows):
        windows.append(test_x[i][:,j*stride:j*stride+input_time_length])
        #The LSTM will be trained on 3 contiguous crops of trials, as in, 3 continuous windows will be fed to LSTM
        if len(windows)==3:
            trials.append(windows)
            labels.append(test_y[i])
            windows=[]
trials=np.array(trials)
labels=np.array(labels)
del windows,test_x,test_y
features=[]
for i in range(len(trials)):
    out=network.predict(trials[i])
    features.append(out)
features=np.asarray(features)
del trials,out
scipy.io.savemat("E:/test_features.mat",{"x":features,"y":labels})
del features,labels

In [5]:
import scipy
import numpy as np
inputs=scipy.io.loadmat("E:/train_features.mat")
X=inputs["x"]
y=inputs["y"].squeeze()
_,t,f=X.shape
del inputs

In [6]:
class SimpleModel(torch.nn.Module):
  def __init__(self,input_features):
    super().__init__()
    self.lstm = torch.nn.LSTM(input_size=input_features, hidden_size=50, batch_first=True)
    self.fc = torch.nn.Linear(50, 2)
    self.tanh = torch.nn.Tanh()
    self.softmax = torch.nn.LogSoftmax(dim=1)

  def forward(self, inputs):
    _, (h1_T,_) = self.lstm(inputs)
    h2=self.tanh(h1_T.squeeze())
    h3 = self.fc(h2)       # inplace of h2[-1,:,:] we can use h2_T. Both are identical
    output = self.softmax(h3)
    return output
model = SimpleModel(f)

In [2]:
inputs=scipy.io.loadmat("E:/test_features.mat")
test_X=inputs["x"]
test_y=inputs["y"].squeeze()
test_set=Dataset(test_X,test_y)

In [10]:
monitor = lambda net: any(net.history[-1, ('valid_accuracy_best','valid_f1_best','valid_loss_best')])
cp=Checkpoint(monitor=monitor,dirname='model',f_params='LSTMbest_param.pkl',f_optimizer='LSTMbest_opt.pkl',f_history='LSTMbest_history.json')
classifier = braindecode.EEGClassifier(
        model,
        criterion=torch.nn.NLLLoss,
        optimizer=torch.optim.AdamW,
        train_split=predefined_split(test_set),
        optimizer__lr=init_lr,
        iterator_train__shuffle=True,
        batch_size=batch_size,
        device=device,
        callbacks=["accuracy","f1",cp],
        warm_start=True,
        )
classifier.initialize()

<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=SimpleModel(
    (lstm): LSTM(13400, 50, batch_first=True)
    (fc): Linear(in_features=50, out_features=2, bias=True)
    (tanh): Tanh()
    (softmax): LogSoftmax(dim=1)
  ),
)

In [13]:
#Try deep smac by itself and as feature extractor and determine effectiveness
classifier.fit(X,y=y,epochs=10)

     21            1.0000      0.9999        0.0027            0.7370      0.6551        3.1672        24.6670
     22            0.9992      0.9975        0.0004            0.7265      0.6372        3.2194        28.9960
     23            0.9996      0.9988        0.0027            0.7397      0.6644        3.0769     +  21.5660
     24            0.9994      0.9983        0.0013            0.7286      0.6559        3.3116        22.1279
     25            1.0000      1.0000        0.0001            0.7370      0.6579        3.5245        25.6141
     26            0.9989      0.9966        0.0028            0.7265      0.6461        2.8728        23.8705
     27            0.9996      0.9988        0.0031            0.7360      0.6537        3.1350        26.3645
     28            0.9996      0.9987        0.0006            0.7201      0.6308        3.3381        25.7681
     29            0.9997      0.9991        0.0008            0.7296      0.6454        3.4281        24.3360
 

<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=SimpleModel(
    (lstm): LSTM(13400, 50, batch_first=True)
    (fc): Linear(in_features=50, out_features=2, bias=True)
    (tanh): Tanh()
    (softmax): LogSoftmax(dim=1)
  ),
)

In [14]:
out=classifier.predict(test_X)
accuracy=np.mean(out==test_y)
print(f"Accuracy:{accuracy}")
tp=np.sum(out*test_y)
precision=tp/np.sum(out)
recall=tp/np.sum(test_y)
f1=2*precision*recall/(precision+recall)
print(f"F1-Score:{f1}") 

Accuracy:0.7343915343915344
F1-Score:0.65283540802213
